In [17]:
from  fuzzywuzzy import fuzz
import re

fuzz.WRatio('activos corrientes', 'Activos Corrrientes ') # 80 treshold de lavenstein score  # ver bien que metrica usar 90 en wratio por ahi


86

In [7]:
from img2table.document import PDF
from img2table.ocr import PaddleOCR

src='Estados_financieros_(PDF)93834000_202012.pdf'
pdf = PDF(src, 
          pages=[0, 2],
          detect_rotation=False,
          pdf_text_extraction=True)

In [8]:
ocr = PaddleOCR(lang="en")
# Table identification and extraction
pdf_tables = pdf.extract_tables(ocr=ocr)

OSError: Tesseract not found in environment. Check variables and PATH

In [97]:
from tabula import read_pdf
import pandas as pd
import os

class BasePDFclass:
    def __init__(self,pdf_path):
        self.pdf_path = pdf_path
    
    def extract_tables(self):
        """ Método a implementar que entregue todas las tablas en formato de una ista de dataframes
        """
        raise NotImplementedError

class Image_PDF_extractor():
    def __init__(self):
        pass

    def exctract_tables(self):
        pass



class Text_PDF_extractor():
    def __init__(self):
        pass
    def extract_tables(self,pdf_path,pages="all"):
        df = read_pdf(pdf_path,pages=pages)
        return(df)


class PDF_tables(BasePDFclass):
    """Class that exctract all the tables of the pdf"""
    
    def __init__(self, pdf_path):
        super().__init__(pdf_path)
        self.text_extractor=Text_PDF_extractor()
        self.image_exctractor=Image_PDF_extractor()

        current_dir = os.getcwd()
        self.csv_path=os.path.join(current_dir, "data","industrydata","pdf","csv")
        self.excel_path=os.path.join(current_dir, "data", "industrydata", "pdf", "excel")

        os.makedirs(self.excel_path, exist_ok=True)
        os.makedirs(self.csv_path, exist_ok=True)



    def extract_tables(self,pages="all"):
        #df_images=self.image_exctractor.extract_tables(pages=pages)
        df_text=self.text_extractor.extract_tables(self.pdf_path,pages=pages)
        #hacer un merge de las dos listas...
        # ojo que en el merge es probable que se repitan las dos tablas
        self.df_list=df_text
        print(f"Extracted {len(self.df_list)} tables")

    def search_concept(self,concept=None): # quizas hacer uno de si es el titulo o no...
        """ Search for a concept in the dataframes, if given, returns a list of dataframes with the given concept
            quizas pasar todo a minuscula...  o a regex
        """ 
        if concept==None:
            return(self.df_list)
        
        match_list=[]
        for df in self.df_list:

            df_lower = df.map(lambda x: x.lower() if isinstance(x, str) else x) # uses lower case to match

            is_string_present = df_lower.isin([concept]).any().any()
            if is_string_present:
                match_list.append(df)
        return(match_list)
    
    def preprocess_data(self,df_list,function=None):
        "quizas implementar funciones propias por empresa..."

        for i,df in enumerate(df_list):

            df = df.apply(lambda x: x.astype(str).str.replace('.', ''))
            df = df.apply(lambda x: x.astype(str).str.replace('(', ''))
            df = df.apply(lambda x: x.astype(str).str.replace(')', ''))
            df.dropna(how="all",inplace=True)
            df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
            df_list[i]=df

        return(df_list)
    
    def tables_to_excel(self,filename,concept=None):
        
        match_list=self.search_concept(concept)
        df_preprocess_list=self.preprocess_data(match_list)
        print(len(df_preprocess_list))
        filepath=os.path.join(self.excel_path, f"{filename}.xlsx")

        with pd.ExcelWriter(filepath) as writer:
            for  i,df in enumerate(df_preprocess_list):
                df.to_excel(writer,sheet_name=f"sheetname_{i}")
        pass
    
    def download_table(self,path,table_name): # ver como seleccionar una tabla para descargar... 
        raise NotImplementedError


In [98]:
pdf_path='Análisis_Razonado92434000_202212.pdf'
pdf_tables_instance=PDF_tables(pdf_path)
pdf_tables_instance.extract_tables()
#df_list=pdf_tables_instance.search_concept()
pdf_tables_instance.tables_to_excel(filename="razonados_besalco")

c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


Extracted 101 tables
101


In [82]:
len(df_list)

101

In [74]:
df=df[0]

In [85]:
for i,df in enumerate(df_list):
    print(i)
    df = df.apply(lambda x: x.astype(str).str.replace('.', ''))
    df = df.apply(lambda x: x.astype(str).str.replace('(', ''))
    df = df.apply(lambda x: x.astype(str).str.replace(')', ''))
    df.dropna(how="all",inplace=True)
    df = df.map(lambda x: x.lower() if isinstance(x, str) else x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [87]:
df_list[21]

,Total Escriturado,Unnamed: 0,Unnamed: 1,31-12-2022,Unnamed: 2,Unnamed: 3,Unnamed: 4,31-12-2021,Unnamed: 5
0,NaN,Rango UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(Proyectos Propios),NaN,M$,NaN,Unidades,NaN,M$,NaN,Unidades
2,Nacional:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0 - 1.000,NaN,-,-,NaN,NaN,-,-
4,NaN,1.001 - 2.000,NaN,-,-,NaN,NaN,-,-
5,NaN,2.001 - 4.000,4.376.357,NaN,48,NaN,7.410.439,NaN,93
6,Casas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,4.001 - 6.000,NaN,-,-,NaN,NaN,-,-
8,NaN,6.001 - 9.000,NaN,-,-,NaN,NaN,-,-
9,NaN,sobre 9.000,9.980.782,NaN,24,NaN,13.197.703,NaN,36
